#### 1.2.1 _Programming Exercise:_ Compute 3-of-3 MuSig public key

In [ ]:
# Compute key pairs
privkey1, pubkey1 = generate_key_pair(sha256(b'key0'))
privkey2, pubkey2 = generate_key_pair(sha256(b'key1'))
privkey3, pubkey3 = generate_key_pair(sha256(b'key2'))
pubkeys = [pubkey1, pubkey2, pubkey3]

# Compute key challenges
# Method: use generate_musig_key() on the list of pubkeys.
# generate_musig_key() returns a challenge map and the aggregate public key.
c_map, pubkey_agg = generate_musig_key(pubkeys)
print("Aggregated Public Key is {}\n".format(pubkey_agg.get_bytes().hex()))

# Multiply key pairs by challenge factor
privkey1_c = privkey1 * c_map[pubkey1]
privkey2_c = privkey2 * c_map[pubkey2]
privkey3_c = privkey3 * c_map[pubkey3]
pubkey1_c = pubkey1 * c_map[pubkey1]
pubkey2_c = pubkey2 * c_map[pubkey2]
pubkey3_c = pubkey3 * c_map[pubkey3]

print("Tweaked privkey1 is {}".format(privkey1_c))
print("Tweaked privkey2 is {}".format(privkey2_c))
print("Tweaked privkey3 is {}".format(privkey3_c))

assert privkey1_c.secret == 11074518666908895565340355428880073686179055674059540498443624547135185713712
assert privkey2_c.secret == 50237208753018228458023990233311512991622479214470726884796884520905307424357
assert privkey3_c.secret == 8793398595099670529210619762464620131014719145314898331758207125003563925169

print("\nSuccess!")

#### 1.2.2 _Programming Exercise:_ Compute 3-of-3 MuSig nonce

In [ ]:
# Generate nonces and nonce points
# We set the nonces manually here for testing purposes, but usually we'll call generate_schnorr_nonce()
# to generate a random nonce point
# Method: generate_schnorr_nonce() with no argument generates a random nonce
k1 = ECKey().set(101)
k2 = ECKey().set(222)
k3 = ECKey().set(333)
test_k1 = ECKey().set(k1.secret)
test_k2 = ECKey().set(k2.secret)
test_k3 = ECKey().set(k3.secret)

# Method: use get_pubkey() to get the associated nonce point.
R1 = k1.get_pubkey()
R2 = k2.get_pubkey()
R3 = k3.get_pubkey()

# Round 1: Generate nonce point commitments and exchange them
# Method: use sha256() on the nonce point. sha256() takes a bytes object, so extract the bytes from the nonce point.
R1_digest = sha256(R1.get_bytes())
R2_digest = sha256(R2.get_bytes())
R3_digest = sha256(R3.get_bytes())

# Round 2: Exchange the nonce points. Each participant verifies that the nonce point commitment matches the nonce point.
assert R1_digest.hex() == "aa5d4a40c1843456534d75f7246b9cbf0f825a36113102e76193f80e6c652c72"
assert R2_digest.hex() == "6a78f2c6ca3bf1364f0575a02ef334d176df2051084c0e3062ec2af79b5b406a"
assert R3_digest.hex() == "bc2c379a7b6ad82b40a7cf280697505ccc039370e50eb155324dcac3b5faa6a5"

# Aggregate nonces
# Tip: Add the individual nonce points together. If the aggregated nonce is not a quadratic residue
# then negate the aggregate nonce and individual nonce scalars.
R_agg = sum([R1, R2, R3])
if jacobi_symbol(R_agg.get_y(), SECP256K1_FIELD_SIZE) != 1:
    R_agg.negate()
    k1.negate()
    k2.negate()
    k3.negate()

print("Individual nonce scalars:\n\t{}, \n\t{}, \n\t{}.\n".format(k1, k2, k3))
print("Aggregate nonce point: {}\n".format(R_agg))

# Test your solution against the aggregate_schnorr_nonces() helper function.
# aggregate_schnorr_nonces() aggregates the nonces and returns whether the individual nonces need to be negated.
test_R_agg, negated = aggregate_schnorr_nonces([R1, R2, R3])
if negated:
    test_k1.negate()
    test_k2.negate()
    test_k3.negate()

assert R_agg == test_R_agg
assert k1 == test_k1
assert k2 == test_k2
assert k3 == test_k3

print("Success!")

#### 1.2.3 _Programming exercise:_ Compute aggregated MuSig signature

In [ ]:
msg = sha256(b'transaction')

# Generate partial signatures
# Method: use sign_musig() with:
#     - individual (tweaked) privkey
#     - individual nonce scalar
#     - aggregate nonce point
#     - aggregate pubkey
#     - msg
s1 = sign_musig(privkey1_c, k1, R_agg, pubkey_agg, msg)
s2 = sign_musig(privkey2_c, k2, R_agg, pubkey_agg, msg)
s3 = sign_musig(privkey3_c, k3, R_agg, pubkey_agg, msg)

print("Partial signatures:\n\t{}\n\t{}\n\t{}\n".format(s1, s2, s3))

# Aggregate signatures
# Method: use aggregate_musig_signatures with list of individual signatures and the aggregate nonce point
sig_agg = aggregate_musig_signatures([s1, s2, s3], R_agg)
print("Aggregate signature:\n\t{}\n".format(sig_agg.hex()))

# Verify signature against aggregate public key
assert pubkey_agg.verify_schnorr(sig_agg, msg)
print("Success! Signature verifies against aggregate pubkey")